<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr2/blob/main/Taranov%20Anatoly/InspectrumClinicRSgr2_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ТЕХНИЧЕСКОЕ ЗАДАНИЕ



Рекомендательная система для профпатолога (INSPECTRUM CLINIC)

Цель: Разработать прототип системы, реализующей функцию рекомендательной системы для врача профпатолога.

Принцип работы: Система должна помогать врачу вынести решение о том годен, не годен или ограниченно годен человек на определенной должности. В датасете целевым столбцом для предсказания нейросетью является столбец под названием: “ЗаключениеМК”. Система может дополнять вердикт мнением, почему человек не годен, или почему ограниченно годен.

Требования интерфейсу: Минимальный уровень реализации: notebook google colaboratory. Требуется реализовать простой веб-интерфейс для загрузки анализируемого моделью нейросети набора данных и удобным выводом результатов вынесенного решения.

Дополнительно: передача продвинутой системы веб-интерфейса, дополняющей вердикт мнением, почему человек не годен, или почему ограниченно годен.

Требования к тестированию, метрикам качества: Оценка результатов работы системы производится Заказчиком на тестовом наборе Заказчика. Метрика точности - точность рекомендации должна быть не ниже 95%

# Загрузка библиотек и датасета

In [ ]:
# Импорт библиотек и инструментов
from google.colab import files              # Импортируем библиотеку files для работы с файлами
import re                                   # Регулярные выражения
import pandas as pd                         # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt             # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import seaborn as sns                       # Импортируем модуль seaborn библиотеки matplotlib для построения графиков
import datetime                             # Импортируем модуль datetime для обработки данных в формете даты
import numpy as np                          # Импортируем библиотеку numpy

%matplotlib inline
import gdown
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

In [ ]:
                                            # Для загрузки данных используем drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/БазыCSV/InClinic/датасет профпатолог часть.xlsx')       # Загрузка файла из диска в colab

In [ ]:
#data2 = pd.read_csv('neiro_06-23.csv', sep = ';', on_bad_lines='skip', engine='python' )

# Анализ формы и состава данных

В ходе анализа предоставленной базы данных выяснилось что достаточно большая часть содержит информацию не влияющую на выполнение задачи по техническому заданию. Принято решение создания датафрейма на позициях необходимых для решения задачи.


---

Запускать только при необходимости анализа

In [ ]:
# путь до папки, содержащей файлы (замените на ваш путь)
folder_path = '/content/drive/MyDrive/Работа в УИИ/Профпатолог стажировка (старички) 01.04/datasets/24.06'

# получаем список файлов в папке
file_list = os.listdir(folder_path)

# последовательно выводим содержимое каждого .csv файла
for file_name in file_list:
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        print("Contens of ", file_name, ":")
        try:
            df = pd.read_csv(file_path, sep=',')
        except:
            try:
                df = pd.read_csv(file_path, sep=';')
            except:
                print("Could not determine separator")
                continue
        display(df) # выводим содержимое файла в виде таблицы

In [ ]:
print(data.shape)                            # Форма и размер
print(data.size)                             # Колличество элементов
data.isna().sum().sum()                      # Вычислит общее количество пропусков во всей таблице
#data.info()                                 # Сводная информация
#data.head()                                 # Вывод первых 5 строк таблицы

(1982, 4587)
9091434


8106978

In [ ]:
data.describe(include=['O'])                  # Статистика

,ДокументПрохождениеМедосмотра,Клиент,КлиентПол,КлиентПолПредставление,ТипМедосмотра,ТипМедосмотраПредставление,Контрагент,Цех,Профессия,ЗаключениеМК,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
count,1982,1982,1982,1982,1982,1982,1982,802,1982,1575,...,1,1,1,1,1,1,1,1,1,1
unique,1915,1911,2,2,2,2,177,183,690,9,...,1,1,1,1,1,1,1,1,1,1
top,e334cb0e-55a6-11ed-8108-0cc47aab8067,116fe6a1-455f-11ed-8107-0cc47aab8067,Мужской,Мужской,Периодический,Периодический,173d6ba7-12fe-11e7-80d3-708bcd7f83cd,Без отдела,Водитель,Годен,...,2934f2d1-8e1f-11ec-80fe-0cc47aab8067,не подлежит обследованию,на ком в горле.,на ком в горле.,C,C,Рекомендации: лекарственные препараты принимат...,Рекомендации: лекарственные препараты принимат...,девиация левого глаза к носу до 25 градусов,девиация левого глаза к носу до 25 градусов
freq,2,3,1451,1451,1981,1981,162,63,55,1434,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
data.info([0,-1])                              # Анализ состава информации в столбцах

In [ ]:
# Выведем уникальные значения для данной колонки
#display(data['Профессия'].unique())

In [ ]:
# Индекс в таблице датафрейма 12
#n = 12
# Значения каждого столбца
#for i in range(len(data.values[n])):
#   print('{:>2}  {}'.format(i, data.values[i][n]))

In [ ]:
# Выведем уникальные значения для данной колонки
display(data['ЗаключениеМК'].unique())

array(['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах',
       'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден',
       'ОграниченноГоден', nan, 'машинист бульдозера', 'Газоспасатель',
       ' Газоспасатель'], dtype=object)

# Отбор столбцов с целевыми данными

* Отбор столбцов - персона/профессия/статус:
"КлиентДатаРождения",  "КлиентПол",  "ТипМедосмотраПредставление", "ПсихОсвидетельствование",  "Цех", "Профессия", "ЗаключениеМК", "ЗаключениеМКПредставление",  "ДиспансернаяГруппаПредставление",
"ВредныеФакторы".

* Отбор столбцов по специализациям врачей
 с целевой информацией - Специализация/обследование/состояние:
"МКБ101",  "МКБ102", "МКБ103",  "МКБ104", " Представление", "ДиагнозПредставление",   "Диагноз_ЗначениеПредставление",
"ПрофнепригодностьВременнаяПредставление",  "ПрофнепригодностьВременнаяПредставление1", "ПрофнепригодностьВременная1","Анамнез_ЗначениеПредставление",  "Профессиональное1", "ЖалобыДополнение", "Рекомендация1", "Заключение".


In [ ]:
# Создание копии таблицы
df = data.copy()

In [ ]:
# Список столбцов с персональными данными
person = ['КлиентДатаРождения', 'КлиентПол', 'Профессия','ВредныеФакторы',
          'ПсихОсвидетельствование', 'ЗаключениеМК', 'ЗаключениеМКПредставление']
def person_columns(df):
    person = []
    for column in df.columns:
                columns_selected.append(df[column])
    return df[person]

person_dat = df[person]

In [ ]:
words = {"МКБ101", "МКБ102","МКБ103", "МКБ104",  "Представление",
         "ПрофнепригодностьВременнаяПредставление","ПрофнепригодностьВременнаяПредставление1",
         "ПрофнепригодностьВременная1","ЖалобыДополнение","Заключение"}
def find_columns(df, words):
    '''
    Функция для поиска столбцов в датафрейме df,
    которые содержат заданные слова из списка words.
    Аргументы:
    - df: датафрейм, в котором нужно искать столбцы
    - words: список слов, которые нужно найти в названиях столбцов
    Возвращает:
    - новый датафрейм, состоящий только из целевых столбцов
    '''
    #разделяем названия столбцов на отдельные слова
    column_words = [s.split('_') for s in df.columns]
    # ищем вхождения каждого слова из списка words
    # в каждом списке слов из column_words
    word_indices = {}
    for i, cw in enumerate(column_words):
        for word in words:
            if word in cw:
                if i not in word_indices:
                    word_indices[i] = 1
                else:
                    word_indices[i] += 1

    #составляем список индексов столбцов, которые содержат
    #хотя бы одно искомое слово
    index_list = [i for i, count in word_indices.items() if count > 0]

    #возвращаем новый датафрейм, состоящий только из найденных столбцов
    return df.iloc[:, index_list]

diagnosis_dat = find_columns(df, words)


In [ ]:
result = pd.concat([person_dat, diagnosis_dat], axis=1)
health_survey = result
# Преобразуем дату рождения, нам нужен только год.
health_survey['КлиентДатаРождения'] = pd.DatetimeIndex(health_survey['КлиентДатаРождения']).year.astype(str)
# Извлечь год из даты и сохранить результаты как числовые значения
health_survey['КлиентДатаРождения'] = pd.DatetimeIndex(health_survey['КлиентДатаРождения']).year
# Преобразовать числовые значения года в строковый формат
health_survey['КлиентДатаРождения'] = health_survey['КлиентДатаРождения'].astype(str)
print(health_survey.shape)
#health_survey.head(1)

(1982, 214)


In [ ]:
# Удаляем полностью пустые столбцы
health_survey.dropna(axis=1, how='all', inplace=True)
# Удаляем дубликаты столбцов
health_survey = health_survey.loc[:,~health_survey.columns.duplicated()]
# Удаляем полностью пустые строки
health_survey.dropna(axis=0, how='all', inplace=True)
# Доля пропусков от общего количества элементов
health_survey.head(2).isna().sum() / health_survey.head(2).shape[0]
# Вычислим общее количество пропусков во всей таблице
health_survey.head(2).isna().sum().sum()
proc = health_survey.head(2).isna().sum().sum() / health_survey.head(2).size
# Вычислим общее количество пропусков в процентах во всей таблице
print('{:.1%}'.format(proc))

89.3%


In [ ]:
# Замена всех пропусков на слово "Нет данных"
health_survey.fillna("Нет данных", inplace=True)
# Форма и размер
print(health_survey.shape)
# Колличество элементов
print(health_survey.size)
health_survey.head(1)

(1982, 214)
424148


,КлиентДатаРождения,КлиентПол,Профессия,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,Рентгенология1_Заключение_ЗначениеПредставление,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
0,1976,Женский,Ведущий специалист по организации обуения,11.4,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,без патологий,без патологий,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
health_survey.to_csv('/content/drive/MyDrive/БазыCSV/InClinic/health_survey.csv', index = False)

# Классификация условий труда



Согласно действующему законодательству, вся трудовая деятельность условно
подразделяется на 4 категории, каждая из которых основывается на степени факторов риска для здоровья и жизни:

---

* Оптимальный — на территории и во внутренних помещениях сохраняется и
поддерживается здоровый микроклимат, положительно влияющий на производительность труда;

* Допустимый — сохраняются нормальные условия, уровень вредных факторов не превышает допустимых норм;

* Вредный — превышаются допустимые нормы, что причиняет вред здоровью человека;

* Опасный — условия труда способны повлечь причинение тяжкого вреда здоровью, иногда несут угрозу жизни.

---

В свою очередь, вредные и опасные производства подразделяются между собой на 4 степени тяжести:

1 Изменения, начавшиеся в организме человека, носят обратимый характер и обычно проявляются после завершения трудовой деятельности. Такие недуги на медицинском сленге называются «профессиональными заболеваниями»;

2 Патологические изменения проявляются более выражено и часто приводят к временной потере трудоспособности (человек регулярно уходит на больничный). Здесь наиболее часто развиваются хронические недуги, вызванные профессиональной деятельностью;

3 В организме протекают необратимые процессы, способные повлечь частичную потерю трудоспособности;

4 Происходят тяжелые функциональные расстройства внутренних органов и систем, что в запущенных случаях приводит к присвоению нерабочей группы инвалидности.


---


К сожалению "СОУТ"(специальная оценка условий труда) пока не располагает системой привязки категории рисков к группам профессии, к примеру ISCO-08, или иные подобные классификаторы.
 Единых методических указаний по управлению профессиональными
рисками пока не существует. Одна из причин - огромное разнообразие профессий и рисков. В одну только Систему стандартов безопасности труда (ССБТ) по состоянию на 2020 год
входит около 400 документов.

---

ISCO-08 делит рабочие места на 10 основных групп:

* Менеджеры
* Профессиональный
* Техники и младшие специалисты
* Работники канцелярской поддержки
* Работники сферы обслуживания и продаж
* Квалифицированные работники сельского, лесного и рыбного хозяйства
* Работники, связанные с ремеслами
* Операторы заводов и станков, а также монтажники
* Элементарные профессии
* Профессии в вооруженных силах


# Анализ данных в отобранных столбцах, отбраковка.

По результату анализа данных в отобранных столбцах остаются только столбцы содержащие условно-полноценную информацию небходимую для обучения "НС"

person - КлиентДатаРождения, КлиентПол, Профессия, ВредныеФакторы,
ПсихОсвидетельствование, ЗаключениеМК,  ЗаключениеМКПредставление.

words - МКБ101, МКБ102, МКБ103, МКБ104,  Представление, ПрофнепригодностьВременнаяПредставление,
ПрофнепригодностьВременнаяПредставление1, ПрофнепригодностьВременная1, ЖалобыДополнение, Заключение.


---

Запускать только при необходимости анализа

Персона

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if 'КлиентПол' in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['Мужской', 'Женский']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if 'Профессия' in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
unique_values

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if 'ВредныеФакторы' in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['5.1,6.2,11.1,11.4', '1.50,3.4,4.4', '4.3.2,11.1,11.3,11.4,12,18.1,18.2', '26', 'Годен к работе в указанных условиях', '1.34.1,1.8.1.1,1.29.1', '8,11.1,18.2', '3.1,4.3.1,4.4,5.1', '5.1,6.2,10,11.1,11.4', '10,11.4,11.1', '1.1,1.8.1.1', '4.3.2,6.1,11.1', '1.14.2,1.37.1,1.45.1,1.49.14,13', '18.1', '1.29.2,4.3.2,11.1,11.4,13', '4.4,11.1,11.4,15,18.1,18.2', '4.3.2,11.1,11.3,11.4,18.1,18.2,8', '4.2.5,4.7,5.1,6.2,11.1', '6.2,11.3,3.1.7,4.2.1,4.4,5.1', '1.1,1.8.1.1,1.8.1.2,1.8.2.1,1.8.2.2,1.8.3.1,1.8.3.2,4.2.5', '4.4,18.1,3.1,11,5.1', '4.2.5,4.4', '1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.34.1,1.37.1,1.50', '3.1.7,4.2.1,4.7,5.1,6.1,11.1,12,13', '18,18.1,18.2', '1.30.1,1.45.1,5.1', '4.3.1,10,11.1', '11.3', '3.1.2,4.4,15', '1.3,1.14.2,5.1', '4.2.5,1.29.2,4.7,4.4,4.1,6.2,11.1', '13,11.1,11.4', '11.1,18.2,4.3.2,4.4', '4.3.2,4.4', '4.1,4.2.5,4.7,5.1,11.1', '4.7,11.1,11.4', '3.1.2,1.19.1,4.3.1,15,1.50,6.1', '4.2.5,4.3.1,4.4,4.7,4.8,4.9,5.1,6,9', '1.1,5.1,6.1', '1.14.2,1.30.1,4.3.1,4.4,5.1,6.1,6.2,15', '1.8.

Результаты обследования

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if 'ПсихОсвидетельствование' in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
unique_values

[False, True]

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if 'ЗаключениеМК' in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['НуждаетсяВДообследованииИЛечении', 'машинист бульдозера', ' Газоспасатель', 'Газоспасатель', 'Временно не годен', 'ГоденБезРаботНаВысотах', 'Годен к работе в указанных условиях', 'ОграниченноГоден', 'ВременноНегоден', 'ГоденСКоррекциейЗрения', 'Нуждается в дообследовании и лечении', 'Ограниченно годен', 'Годен к работе в указанных условиях (без работ на высотах)', 'Годен к работе в указанных условиях (с коррекцией зрения)', 'тракторист', 'Годен', 'Нет данных']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if 'ЗаключениеМКПредставление' in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['Временно не годен', 'Годен к работе в указанных условиях', 'Нуждается в дообследовании и лечении', 'ГоденСКоррекциейЗрения', 'Ограниченно годен', 'Годен к работе в указанных условиях (без работ на высотах)', 'Годен к работе в указанных условиях (с коррекцией зрения)', 'тракторист', 'Годен', 'Нет данных']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "Представление" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
unique_values

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "МКБ101" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['I69.4', 'K82.8', 'K03.66', 'Q12.0', 'I11.9', 'N83.2', 'K08.2', 'K04.5', 'I25.9', 'K25.7', 'H53.0', 'F41.0', 'K29.8', 'L20.8', 'I25.2', 'F98.5', 'S68.2', 'I83.9', 'E10.9', 'K86.1', 'K80.1', 'H50.9', 'D25.9', 'I67.2', 'K29.4', 'H25.1', 'F10.7', 'H90.4', 'Амбулаторная помощь при необходимости', 'E11.9', 'H52.1', 'H11.0', 'K43.6', 'H02.4', 'K25.9', 'M42.1', 'K81.9', 'I70', 'нет', 'Нет данных', 'H52.2', 'G25.0', 'I66.9', 'K00.0', 'Z00.0', 'T90.4', 'N60.2', 'J33.0', 'K02.1', 'H35.0', 'D25', 'J45.9', 'K29.5', 'G58.9', 'K29.6', 'L40.0', 'S51.9', 'N80.0', 'K05.3', 'K29.7', 'H40.1', 'I67.9', 'G99.0', 'K26.7', 'K02.0', 'E11.8', 'T94.0', 'K82.9', 'T92.0', 'H93.0', 'R25.1', 'F06.7', 'G12.1', 'J37.0', 'E14.8', 'H47.2', 'H40.9', 'K80.5', 'J30.1', 'H01.0', 'H91.1', 'K40.9', '[Z00.0] Общий медицинский осмотр_x000D_\n', 'K29.3', 'H25.0', 'чистая', 'H90.5', 'G90.9', 'F06.6', 'G90.8', 'I49.9', 'F10.1', 'F70.0', 'F07.0', 'H90.3', 'H66.1', 'K26.9', 'H52.0', 'K76.0', 'H74.1']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "МКБ102" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['C53.9', 'K05.3', 'H02.4', 'I49.8', 'H17.1', 'M42.1', 'I11.9', 'J45.8', 'H91.1', 'K40.9', 'I67.9', 'H53.0', 'K29.3', 'H25.0', 'K29.8', 'Нет данных', 'K05.4', 'I25.8', 'H52.2', 'K26.7', 'K02.0', 'I25.2', 'D35.2', 'K00.0', 'Z00.0', 'T94.0', 'I49.9', 'H90.3', 'J33.0', 'H52.0', 'K02.1', 'K86.1', 'H50.9', 'D25.9', 'N28.1', 'K29.6', 'H74.1', 'H47.2', 'K44.9', 'H52.1']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "МКБ103" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['I11.9', 'J45.8', 'H52.0', 'I25.8', 'H52.2', 'I25.2', 'K80.2', '[Z00.0] Общий медицинский осмотр_x000D_\n', 'K29.3', 'H90.3', 'H52.1', 'Нет данных']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "МКБ104" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['I11.9', 'H52.0', 'H52.2', 'I25.2', 'K80.2', 'H90.3', 'H52.1', 'Нет данных']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "ПрофнепригодностьВременнаяПредставление" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['БИТУФА_НеВыявлена', 'Временно не годен', 'Не годен', 'Годен, кроме п.', 'Годен', 'Нет данных']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "ПрофнепригодностьВременнаяПредставление1" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['БИТУФА_НеВыявлена', 'Временно не годен', 'Не годен', 'Годен, кроме п.', 'Годен', 'Нет данных']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "ПрофнепригодностьВременная1" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
print(unique_values)



['Временная', 'false', 'Нет данных', 'ХР_ГоденКромеПунктов', 'Постоянная', 'БИТУФА_НеВыявлена']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if "ЖалобыДополнение" in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
unique_values

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if 'Заключение' in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
unique_values

# Корректировка данных



План:

Сделать выборку в колонке 'Профессия' по основному показателю профессии.

Создать списки категорий профессий по должностям с разделением на степени факторов риска.

Степени факторов риска: Оптимальный,  Допустимый,  Вредный,  Опасный.

Присвоить категории профессии индекс факторов риска, создать новую колонку 'класс безопасности' и заполнить соответствующими значениями индексов.

Создать матрицу корреляции 'Профессия' - 'класс безопасности' - 'ВредныеФакторы'.  Анализ.

Создать матрицу корреляции 'Профессия' - 'класс безопасности' - 'ВредныеФакторы' --- 'Возраст' - 'Пол' - 'Представление' - 'Заключение'.  Анализ.

 Ранжирование рисков.



In [ ]:
health_survey = pd.read_csv('/content/drive/MyDrive/БазыCSV/InClinic/health_survey.csv')

In [ ]:
df_p = pd.read_excel('/content/drive/MyDrive/БазыCSV/InClinic/Проф2.xlsx')

In [ ]:
# Создание копии таблицы
df_edit =health_survey.copy()

In [ ]:
df_edit.head(1)

РАБОТА С КОЛОНКОЙ "Профессия" НЕ ЗАКОНЧЕНА

In [ ]:
# Создаём пустой словарь
my_dict = {}

# Проходим по всему датафрейму и заносим данные в словарь
for index, row in df_p.iterrows():
    my_dict[row[0]] = row[1]

# Создаём список ключей для поиска в датафрейме
keys = list(my_dict.keys())

for row in df_edit['Профессия']:   # перебор колонки 'Профессия' из датафрейма df_edit
    for word in df_p.iloc[:,0]:    # поиск вхождения слов из df_p[:, 0] в каждой строке колонки 'Профессия'
        if row.str.contains(word):    # если колонка содержит слово из df_p[:, 0], то
            row['Профессия'] = word # значение в колонке 'Профессия' присваивается соответствующему ключевому слову из df_p

# Создаём новую колонку 'класс безопасности' и помещаем её под индексом [3]
df_edit.insert(3, 'класс безопасности', '0')

# Обновляем новую колонку значениями из словаря, если соответствующий ключ присутствует в выбранных строках
for index, row in selected_rows.iterrows():
    if row['Профессия'] in my_dict.keys():
        df_edit.at[index, 'класс безопасности'] = my_dict[row['Профессия']]

AttributeError: ignored

In [ ]:
# Создаём пустой словарь
my_dict = {}

# Проходим по всему датафрейму и заносим данные в словарь
for index, row in df_p.iterrows():
    my_dict[row[0]] = row[1]

# Создаём список ключей для поиска в датафрейме
keys = list(my_dict.keys())

# Проходим по колонке 'Профессия' и делаем выборку строк, содержащих ключевые слова
selected_rows = df_edit[df_edit['Профессия'].str.contains('|'.join(keys))]

# Создаём новую колонку 'класс безопасности' и помещаем её под индексом [3]
df_edit.insert(3, 'класс безопасности', '0')

# Обновляем новую колонку значениями из словаря, если соответствующий ключ присутствует в выбранных строках
for index, row in selected_rows.iterrows():
    if row['Профессия'] in my_dict.keys():
        df_edit.at[index, 'класс безопасности'] = my_dict[row['Профессия']]

In [ ]:
df_edit.head(1)

,КлиентДатаРождения,КлиентПол,Профессия,класс безопасности,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
0,1976,Женский,Ведущий специалист по организации обуения,0,11.4,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,без патологий,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
import pandas as pd

# Загружаем датафрейм df_p
#df_p = pd.read_csv('path_to_file')

# Задаём список значений из колонки 0 для поиска
list_col_0 = df_p.iloc[:, 0].tolist()

# Импортируем датафрейм df_edit
#df_edit = pd.read_csv('path_to_file')

# Создаём словарь для значений колонок из df_p
my_dict = dict(zip(df_p.iloc[:,0], df_p.iloc[:,1]))

# Создаём список индексов, в которых происходит вхождение слов из списка колонки 0 из df_p
index_list = []
for i in range(len(df_edit)):
    if any(word in df_edit['Профессия'][i] for word in list_col_0):
        index_list.append(i)

# Присваиваем статус ключа значениям колонки 'Профессия', указанным в полученном списке индексов
df_edit.loc[index_list, 'Профессия'] = df_edit.loc[index_list, 'Профессия'].apply(lambda x: my_dict[x])

# Создаём колонку 'класс безопасности' и заполняем её нулями
df_edit.insert(2, 'класс безопасности', 0)

# Присваиваем статус ключа значениям колонки 'класс без


KeyError: ignored

In [ ]:
# Загружаем датафрейм df_p
import pandas as pd
df_p = pd.DataFrame()
my_dict = {}

for index, row in df_p.iterrows():
    my_dict[row[0]] = row[1]

# Задаём список значений из колонки 0 для поиска
list_col_0 = df.iloc[:, 0].tolist()

# Импортируем датафрейм health_survey
df_edit = pd.DataFrame()

# Создаём список индексов, в которых происходит вхождение слов из списка колонки 0 из df_p
index_list = []
for i in range(len(df_edit)):
    if any(word in df_edit['Профессия'][i] for word in list_col_0):
        index_list.append(i)

# Присваиваем статус ключа значениям колонки 'Профессия', указанным в полученном списке индексов
df_edit.loc[index_list, 'Профессия'] = my_dict{:, 0}

# Создаём колонку 'класс безопасности' и заполняем её нулями
df_edit.insert(2, 'класс безопасности', 0)

# Присваиваем статус ключа значениям колонки 'класс безопасности', указанным в полученном списке индексов
df_edit.loc[index_list, 'класс безопасности'] = my_dict{:, 1}


SyntaxError: ignored

In [ ]:
health_survey.loc[index_list, 'Профессия'] = 'active'


In [ ]:
# Создание словаря
dic = {df_p.iloc[i,0]: df_p.iloc[i,1] for i in range(len(df_p))}

In [ ]:
# создание списка слов из колонки 0 датафрейма df_p
words = list(df_p['0'])

# выборка индексов ячеек строк в значениях которых есть слова, соответствующие словам из колонки 0 из df_p
indices = []
for index, row in health_survey.iterrows():
    if any(word in row['Профессия'] for word in words):
        indices.append(index)

# создание словаря my_dict, используя первую колонку как ключ и вторую как значение
my_dict = dict(zip(df_p.iloc[:,0], df_p.iloc[:,1]))

# присвоение


KeyError: ignored

In [ ]:
my_dict = dict(zip(df_p.iloc[:, 0], df_p.iloc[:, 1]))

for index, row in health_survey.iterrows():
    professions = row['Профессия'].split()
    matching_indexes = [i for i, word in enumerate(professions) if any(key in word for key in my_dict.keys())]
    for i in matching_indexes:
        health_survey.at[index, 'статус'] = my_dict.get(professions[i])

health_survey['класс безопасности'] = [my_dict.get(job, 'Unknown') for job in health_survey['Профессия']]


In [ ]:
health_survey

In [ ]:
# создаем пустой словарь "my_dict"
my_dict = {}

# проходимся по всем строкам в датафрейме "df_p"
for index, row in df_p.iterrows():
    # добавляем первую колонку в качестве ключа и вторую колонку в качестве значения словаря
    my_dict[row[0]] = row[1]

for index, row in health_survey.iterrows():
    professions = row['Профессия'].split()
    matching_indexes = [i for i, word in enumerate(professions) if any(key in word for key in my_dict.keys())]
    for i in matching_indexes:
        health_survey.at[index, 'статус'] = my_dict.get(professions[i])


# создаем новую колонку "класс безопасности" в датафрейме "health_survey" и заполняем ее значениями из словаря в соответствии с ключами из колонки "Профессия"
health_survey[2,'класс безопасности'] = ""
for index, row in health_survey.iterrows():
    if index in indexes:
        # присваиваем значение соответствующего ключа из словаря "my_dict"
        health_survey.at[index, 'класс безопасности'] = my_dict[row['Профессия']]

NameError: ignored

In [ ]:
# создаем пустой словарь "my_dict"
my_dict = {}

# проходимся по всем строкам в датафрейме "df_p"
for index, row in df_p.iterrows():
    # добавляем первую колонку в качестве ключа и вторую колонку в качестве значения словаря
    my_dict[row[0]] = row[1]

# создаем список для индексов строк в колонке "Профессия", где есть вхождения из колонки 0 в "df_p"
indexes = []
for index, row in health_survey.iterrows():
    for word in df_p[0]:
        if word in row['Профессия']:
            # добавляем индекс строки в список
            indexes.append(index)

# создаем новую колонку "класс безопасности" в датафрейме "health_survey" и заполняем ее значениями из словаря в соответствии с ключами из колонки "Профессия"
health_survey['класс безопасности'] = ""
for index, row in health_survey.iterrows():
    if index in indexes:
        # присваиваем значение соответствующего ключа из словаря "my_dict"
        health_survey.at[index, 'класс безопасности'] = my_dict[row['Профессия']]


KeyError: ignored

In [ ]:
# Создание словаря
dic = {df_p.iloc[i,0]: df_p.iloc[i,1] for i in range(len(df_p))}

# Прохождение по колонке 'Профессия' в 'health_survey' и присвоение статуса ключа
for i in range(len(health_survey)):
    for j in range(len(df_p)):
        if df_p.iloc[j,0] in health_survey.loc[i,'Профессия']:
            health_survey.loc[i,'Статус'] = dic[df_p.iloc[j,0]]

# Создание колонки 'класс безопасности' и заполнение значений из словаря
health_survey.insert(2, 'Класс безопасности', '')
for i in range(len(health_survey)):
    if health_survey.loc[i,'Профессия'] in dic:
        health_survey.loc[i,'Класс безопасности'] = dic[health_survey.loc[i,'Профессия']]


KeyError: ignored

In [ ]:
import pandas as pd

# Создание словаря из датафрейма df_prof
df_p = pd.DataFrame({'Ключ': df_p.iloc[:, 0], 'Значение': df_p.iloc[:, 1]})
dictionary = dict(zip(df_p['Ключ'], df_p['Значение']))

# Замена значений в колонке 'Профессия' датафрейма 'health_survey' на значения из словаря
professions = health_survey['Профессия'].tolist()
health_survey['Профессия'] = [dictionary.get(prof, prof) for prof in professions]

# Создание колонки 'класс безопасности' и заполнение значениями из словаря
health_survey['класс безопасности'] = [dictionary.get(prof, '') for prof in professions]

health_survey.head(3)


In [ ]:
dictionary

In [ ]:
# Создаем словарь из df_prof
dictionary = dict(zip(df_p.iloc[:, 0], df_p.iloc[:, 1]))

# Проходим по колонке 'Профессия' в 'health_survey' и заменяем значения на значения из списка ключей словаря
for i, value in enumerate(health_survey['Профессия']):
    for key in dictionary.keys():
        if key in value:
            health_survey.loc[i, 'Профессия'] = dictionary[key], inplace=True



In [ ]:
health_survey.head(1)

,КлиентПол,Профессия,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,Рентгенология1_Заключение_ЗначениеПредставление,ПсихиатрияНаркология1_МКБ101,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
КлиентДатаРождения,,,,,,,,,,,,,,,,,,,,,
1976,Женский,Ведущий специалист по организации обуения,11.4,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,без патологий,без патологий,Z00.0,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
# Создаем колонку 'класс безопасности' и заполняем ее значениями из словаря
health_survey.insert(2, 'класс безопасности', '')
for i, value in enumerate(health_survey['Профессия']):
    if value in dictionary.keys():
        health_survey.loc[i, 'класс безопасности'] = dictionary[value]

ValueError: ignored

In [ ]:
health_survey.head(1)

,КлиентПол,Профессия,класс безопасности,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,Рентгенология1_Заключение_ЗначениеПредставление,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
КлиентДатаРождения,,,,,,,,,,,,,,,,,,,,,
1976,Женский,Ведущий специалист по организации обуения,,11.4,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,без патологий,без патологий,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
# Создание словаря из датафрейма df_prof
dictionary = dict(zip(df_p.iloc[:, 0], df_p.iloc[:, 1]))

# Выборка строк из датафрейма health_survey с помощью метода contains


# Добавление новой колонки 'класс безопасности' в датафрейм health_survey
# и заполнение значениями из словаря в соответствии с ключами из колонки 'Профессия'

print(dictionary)

{" 'Авиамеханик '": 2, " 'Авиатехник'": 2, " 'агроном'": 6, " 'администратор'": 1, " 'Анестезиолог'": 2, " 'Ассистент врача'": 3, " 'Барменеджер'": 9, " 'Бригадир'": 8, " 'Бульдозерист',": 8, " 'Бурильщик'": 8, " 'Бухгалтер'": 4, " 'Вальщик леса'": 9, " 'Взрывник'": 8, " 'Воспитатель'": 5, " 'Врач'": 2, " 'Вышкомонтажник'": 8, " 'Газорезчик'": 8, " 'Гардеробщик'": 9, " 'геофизик'": 2, " 'Горничная'": 9, " 'грузчик'": 9, " 'Делопроизводитель'": 9, " 'Дефектоскопист'": 3, " 'Директор'": 1, " 'Диспетчер'": 9, " 'Дозиметрист'": 3, " 'ЖКХ'": 5, " 'Зав.'": 4, " 'Заведу'": 4, " 'Заместитель'": 4, " 'Инженер'": 2, " 'Инспектор'": 3, " 'Испытатель'": 3, " 'Каротажник'": 7, " 'кассир'": 5, " 'Кладовщик'": 9, " 'Комендант'": 9, " 'Комплектовщик'": 9, " 'Кондитер'": 7, " 'Купажист'": 7, " 'курьер'": 9, " 'Кухонный работник'": 9, " 'ЛАБОРАНТ'": 8, " 'Литейщик'": 7, " 'Маляр'": 7, " 'Мастер'": 3, " 'машинист экструдера'": 8, " 'Машинист'": 8, " 'Мед.регистратор'": 4, " 'мед.сестра'": 7, " 'Мед'": 7,

In [ ]:
# Создаём пустой словарь
my_dict = {}

# Проходим по всему датафрейму и заносим данные в словарь
for index, row in df_p.iterrows():
    my_dict[row[0]] = row[1]

# Выборка строк из датафрейма health_survey с помощью метода contains
selected_rows = health_survey[health_survey['Профессия'].str.contains('|'.join(dictionary.keys()))]


In [ ]:
selected_rows

,КлиентПол,Профессия,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,Рентгенология1_Заключение_ЗначениеПредставление,ПсихиатрияНаркология1_МКБ101,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
КлиентДатаРождения,,,,,,,,,,,,,,,,,,,,,


In [ ]:
health_survey.head(1)

,КлиентПол,Профессия,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,Рентгенология1_Заключение_ЗначениеПредставление,ПсихиатрияНаркология1_МКБ101,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
КлиентДатаРождения,,,,,,,,,,,,,,,,,,,,,
1976,Женский,Ведущий специалист по организации обуения,11.4,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,без патологий,без патологий,Z00.0,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
# Добавление новой колонки 'класс безопасности' в датафрейм health_survey
# и заполнение значениями из словаря в соответствии с ключами из колонки 'Профессия'
health_survey.insert(2, 'класс безопасности', selected_rows['Профессия'].map(dictionary))

ValueError: ignored

In [ ]:
health_survey.head(1)

,КлиентПол,Профессия,ВредныеФакторы,класс безопасности,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,Рентгенология1_Заключение_ЗначениеПредставление,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
КлиентДатаРождения,,,,,,,,,,,,,,,,,,,,,
1976,Женский,Ведущий специалист по организации обуения,11.4,нет данных,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,без патологий,без патологий,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
# Создаём пустой словарь
my_dict = {}

# Проходим по всему датафрейму и заносим данные в словарь
for index, row in df_p.iterrows():
    my_dict[row[0]] = row[1]

# Создаём список ключей для поиска в датафрейме
keys = list(my_dict.keys())

# Проходим по колонке 'Профессия' и делаем выборку строк, содержащих ключевые слова
selected_rows = health_survey[health_survey['Профессия'].str.contains('|'.join(keys))]

# Создаём новую колонку 'класс безопасности' и помещаем её под индексом [3]
health_survey.insert(3, 'класс безопасности', 'нет данных')

# Обновляем новую колонку значениями из словаря, если соответствующий ключ присутствует в выбранных строках
for index, row in selected_rows.iterrows():
    if row['Профессия'] in my_dict.keys():
        health_survey.at[index, 'класс безопасности'] = my_dict[row['Профессия']]


In [ ]:
health_survey.head(1)

,КлиентПол,Профессия,ВредныеФакторы,класс безопасности,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,Рентгенология1_Заключение_ЗначениеПредставление,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
КлиентДатаРождения,,,,,,,,,,,,,,,,,,,,,
1976,Женский,Ведущий специалист по организации обуения,11.4,нет данных,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,без патологий,без патологий,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
# Чтение данных в датафрейм
df_p = pd.read_excel('/content/drive/MyDrive/БазыCSV/InspectrumClinic/Проф1.xlsx')

# Преобразование данных в словарь
my_dict = df_p.set_index('Проф').to_dict()['КлУсл']

In [ ]:
df_p = pd.DataFrame{[Проф] : [], [КлУсл] : []}

dictionary = df_p.set_index(df_p.columns[Проф]).to_dict()[df_p.columns[КлУсл]]

print(dictionary)


SyntaxError: ignored

In [ ]:
# датафрейм
df_p = pd_p.DataFrame({[:, 0] : [], [:, 1] : []})

# преобразуем датафрейм в словарь
dictionary = df_p.set_index.iloc[:, 0].to_dict(orient='dict')[:, 1]

# выведем на экран полученный словарь
print(dictionary)


SyntaxError: ignored

In [ ]:
# Чтение данных в датафрейм
df_p = pd.read_excel('/content/drive/MyDrive/БазыCSV/InspectrumClinic/Проф1.xlsx')

# Преобразование данных в словарь
my_dict = df_p.set_index('Проф').to_dict()['КлУсл']


In [ ]:
# создание словаря
prof_dict = my_dict[]
prof_dict = {}

# выборка строк с помощью метода str.contains
selected_rows = health_survey[health_survey['Профессия'].str.contains('|'.join(prof_dict.keys()))].index

# замена значений в колонке 'Профессия' на значения индексов словаря
health_survey.loc[selected_rows, 'Профессия'] = [prof_dict[i] for i in health_survey.loc[selected_rows, 'Профессия']]


SyntaxError: ignored

In [ ]:
# загрузка данных датафреймы 'health_survey' и 'df_p'
health_survey = pd.read_csv('/content/drive/MyDrive/БазыCSV/health_survey.csv', index_col=0)
df_p = pd.read_excel('/content/drive/MyDrive/БазыCSV/InspectrumClinic/Прфессии9кл.xlsx')

# создание списка слов из колонки 0 датафрейма df_p
words = list(df_p.iloc[:, 0])

# проход по колонке 'Профессия' и выбор индексов значений по вхождениям слов из списка
indexes = health_survey[health_survey['Профессия'].str.contains('|'.join(words))].index

# замена значений в колонке 'Профессия' на значения из списка колонки 0 в df_p
health_survey.loc[indexes, 'Профессия'] = df_p.iloc[:, 0].values

# сохранение изменений в файл
health_survey.to_csv('health_survey_up.csv', index=False)


НЕ ЗАПУСКАТЬ - Дежурный код

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer

# считываем данные из файла в датафрейм
#df = pd.read_csv('имя_файла.csv')

# выбираем нужные столбцы из датафрейма
selected_columns = ['', '']
health_survey_selected = health_survey[selected_columns]

# вычисляем корреляцию между столбцами
correlations = health_survey_selected.corr()

# выбираем столбцы с коэффициентом корреляции > 0.6
selected_columns = correlations.columns[correlations.iloc[-1] > 0.6]

# создаём набор текстовых документов, каждый из которых состоит из значений выбранных столбцов
documents = list(health_survey[selected_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1).values)

# преобразуем набор документов в последовательность индексов в матрице Bag of Words
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(documents)


Удаляем строки не имеющие целевых значений в столбце "ЗаключениеМК"

In [ ]:
index_list = health_survey[(health_survey['ЗаключениеМК'] == 'машинист бульдозера') |
 (health_survey['ЗаключениеМК'] == 'Газоспасатель') |
  (health_survey['ЗаключениеМК'] == 'Нет данных')].index.tolist()

In [ ]:
health_survey.drop(index_list, inplace=True)

In [ ]:
# Уникальные значения колонки
display(health_survey['ЗаключениеМК'].unique())

array(['Годен', 'ГоденСКоррекциейЗрения'], dtype=object)

Корректируем данные в колонке " "

ДАЛЕЕ РАБОТА НЕ ПРОИЗВОДИЛАСЬ

# Формирование классов
Ввиду того что задача создаваемой НС носит характер "Рекомендательной системы"
по этому колонки "ЗаключениеМК" для полноценного анализа не достаточно так как данные содержащиеся в этой колонке разбалансированны и мало информативны для целевой рекомендации.

Есть версия что в колонке "ЗаключениеМК" надо рассматривать два класса "Годен" и 'НуждаетсяВДообследовании'. Так для рекомендации необходимо понимание самой рекомендации или вектор. При фомировании y_train для эффективности обучения так же следует расматривать столбцы с уже приобретёнными болезнями и явными диагнозами такие как
'Заключение', "ЖалобыДополнение" в виде отрицательных факторов 'диагноза'  --
 ("Заключения МК").

Так мы устраняем разбалансировку и даём вектор поиска патологий.


In [ ]:
# Импорт библиотек и инструментов
from google.colab import files              # Импортируем библиотеку files для работы с файлами
import pandas as pd                         # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt             # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import seaborn as sns                       # Импортируем модуль seaborn библиотеки matplotlib для построения графиков
import gdown
import numpy as np
import re
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

In [ ]:
health_survey = pd.read_csv('/content/drive/MyDrive/БазыCSV/health_survey.csv', index_col=0)

In [ ]:
# Создание копии таблицы
health_survey_backup = health_survey.copy()
health_survey.head(1)

In [ ]:
health_survey.head(1)

,КлиентПол,Профессия,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Рентгенология1_Заключение_Значение,Рентгенология1_Заключение_ЗначениеПредставление,ПсихиатрияНаркология1_МКБ101,...,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_ПрофнепригодностьВременная1,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_МКБ102,Оториноларингология3_МКБ101,Оториноларингология3_ПрофнепригодностьВременная1,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление
КлиентДатаРождения,,,,,,,,,,,,,,,,,,,,,
1976,Женский,Ведущий специалист по организации обуения,11.4,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,без патологий,без патологий,Z00.0,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:

def adg_p(arg):
    # текущий год - год рождения
    adg = [0]  # создаем список и заполняем нулями
    year_tec = 2023
    arg = int(arg)  # преобразуем строку в число
    # извлекаем год из числа и вычисляем возраст
    adg_year = year_tec - int(re.findall(r'\d{4}', str(arg))[0])
    adg[0] = adg_year  # заменяем нулевой элемент списка на полученный возраст
    return adg

def getAgeVect(adg):
    age_c = []
    # превращаем возраст в категорию возрастной группы
    outClass =int((adg-10)/12)
    outClass = max(0, min(5,outClass))
    # создаем вектор с нужной категорией возраста
    result = np.eye(5)[outClass].astype('int')
    return age_c

# Проверяем функции и выводим результаты
adg = adg_p(2019)
age_c = getAgeVect(adg[0])
age_c

[]

In [ ]:
def Sex_class(arg):
    # Создаем список с одним элементом 0
    sex_c = [0]
    # Если М, то значение 1 (по умолчанию 0 для женского пола)
    if 'М' in arg:
        sex_c = 1
    # Возвращаем список
    return sex_c
# Проверяем функцию с аргументом 'м'
sex_c = Sex_class('Ж')
# Выводим значение
sex_c

[0]

Приводим значения в столбце "ЗаключениеМК" к двум классам:
* класс- Годен
* класс- НуждаетсяВДообследовании

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in health_survey.columns:
    if 'ЗаключениеМК' in column:
        unique_values.extend(health_survey[column].unique())
unique_values = list(set(unique_values))
unique_values

['Годен',
 'Годен к работе в указанных условиях',
 'НуждаетсяВДообследовании',
 'Годен к работе в указанных условиях (с коррекцией зрения)']

In [ ]:
class_list = ['Годен', 'НуждаетсяВДообследовании']

def health_status(health_survey, column_name, class_list):
    for class_name in class_list:
        count = len(health_survey[health_survey[column_name] == class_name])
        print(f'Количество записей для класса {class_name}: {count}')

# Удаление строк без заключения
mask = health_survey['ЗаключениеМК'].str.contains('|'.join(class_list)) > 0
condition = health_survey[mask]

# Сокращение классов до 3
health_survey['ЗаключениеМК'].replace('ГоденСКоррекциейЗрения', 'НуждаетсяВДообследовании', inplace=True)
health_survey['ЗаключениеМК'].replace('ГоденБезРаботНаВысотах', 'НуждаетсяВДообследовании', inplace=True)
health_survey['ЗаключениеМК'].replace('НуждаетсяВДообследованииИЛечении', 'НуждаетсяВДообследовании' , inplace=True)
health_survey['ЗаключениеМК'].replace('Нет данных', 'НуждаетсяВДообследовании', inplace=True)

# количество записей по классам
health_status(health_survey, 'ЗаключениеМК', class_list)

# Функция для определения статуса здоровья
def health_stat(condition):
    out = [0, 0]
    if ('Годен' in condition):
        out[0]=1
    if ('НуждаетсяВДообследовании' in condition):
        out[1]=1

    # Присваиваем список 'out' переменной 'condition_ohe'
    condition_ohe = out
    return condition_ohe
# Пример вызова функции
health_stat('годен')
condition_ohe = health_stat(condition)
print(condition_ohe)

In [ ]:
# Функция для получения диагнозов из данных
def get_diagnosis(data):
    diagnosis = []
    for val in data:
        if isinstance(val[0], str):
            diagnosis.append(val[0])
    diagnosis = np.array(diagnosis)
    return diagnosis

# Получение диагнозов из данных
diagnosis = get_diagnosis(health_survey.values)

# Создание и обучение токенизатора для диагнозов
max_words_count = 2000
tokenizer = Tokenizer(num_words=max_words_count, filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', lower=True, split=' ', oov_token='unknown', char_level=False)
tokenizer.fit_on_texts(diagnosis)
diagnosis_indexes = tokenizer.texts_to_sequences(diagnosis)
diagnosis_01 = tokenizer.sequences_to_matrix(diagnosis_indexes)


ДАЛЕЕ РАБОТА НЕ ПРОИЗВОДИЛАСЬ

# Формирование выборок для обучения

In [ ]:

#age_c = getAgeVect(adg[0])
# Функция для получения всех параметров из данных
def get_all_parameters(val):
    age_c = getAgeVect(adg)
    age_vector = age_c(val)
    sex_class = get_sex_class(val[2])
    diagnosis_result = tokenizer.texts_to_matrix([val[0]])

    parameters = age_vector + [sex_class] + list(diagnosis_result[0])
    return parameters

In [ ]:

#age_c = getAgeVect(adg[0])
# Функция для получения всех параметров из данных
def get_all_parameters(val):
    adg = adg_p(val)
    age_c = getAgeVect(adg[0][0])
    sex_class = get_sex_class(val[2])
    diagnosis_result = tokenizer.texts_to_matrix([val[0]])

    parameters = age_c + [sex_class] + list(diagnosis_result[0])
    return parameters

In [ ]:
# Функция для получения обучающих данных в формате 0/1
def get_01_data(data):
    x_train = []
    y_train = []

    for val in data:
        y = health_stat(val)
        x = get_all_parameters(val)
        x_train.append(x)
        y_train.append(y)

    x_train = np.array(x_train)
    y_train = np.array(y_train)

    return x_train, y_train

x_train, y_train = get_01_data(health_survey.values)

In [ ]:
health_survey_subset = health_survey.iloc[:, [1,2,3,4,5]] # Выбор столбцов с нужными индексами
text_data = health_survey_subset.to_string(header=False, index=False) # Преобразование в строку
tokens = text_data.lower().split() # Приведение к нижнему регистру и токенизация


In [ ]:
# Создание экземпляра класса CountVectorizer
vectorizer = CountVectorizer()

# Преобразование токенов в матрицу Bag of Words
bow_matrix = vectorizer.fit_transform(tokens)

# Преобразование в разреженный массив NumPy
sparse_bow_matrix = bow_matrix.toarray()


In [ ]:
print(sparse_bow_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# Функция перевода классов в OHE
def Y_to_OHE(df):
  y_list_key = list(df.ЗаключениеМК.values)
  # print (y_list_key)
  y_list = []
  for CD in y_list_key:
    y_list.append(CLASS_DICT[CD])
  # print (y_list)

  y_data = np.array(y_list)            # Перевод общего списка меток класса в numpy-массив

  y_res = utils.to_categorical(y_data, CLASS_COUNT)
  return y_data, y_res


In [ ]:
y_data_01, y_train_01 = Y_to_OHE(dffix5_cut)
print (y_data_01[100])
print (y_train_01[100])

y = np.argmax(y_train_01, axis=-1)
print (y[100])

2
[0. 0. 1.]
2


# Эксперименты обучения

In [ ]:
health_stat = pd.read_csv('/content/health_stat.csv')

In [ ]:
from tensorflow.keras import utils
loss, acc = search.evaluate(x1_test, y_test, verbose=0) #classification
mae, _ = search.evaluate(x1_test, y_test, verbose=0) #regression
y_predictions = search.predict(x1_test)
model = search.export_model()
model.summary()
utils.plot_model(model, show_shapes=True, show_layer_names=True, dpi=60, show_layer_activations=True, show_trainable=True)

In [ ]:
# Обучаем модель полученными данными
model = Sequential()
model.add(BatchNormalization(input_shape=(xTrain01.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1000, activation='tanh'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(lr=1e-5), loss='mse', metrics=['mae'])

history = model.fit(xTrain01,
                    yTrain,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.15,
                    verbose=2)

plt.plot(history.history['mae'],
         label='Средняя абсолютная ошибка на обучающем наборе')
plt.plot(history.history['val_mae'],
         label='Средняя абсолютная ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Средняя абсолютная ошибка')
plt.legend()
plt.show()

In [ ]:
import autokeras as ak
search = ak.StructuredDataClassifier(max_trials=15)
search.fit(x=x1_train, y=y_train, verbose=1)

In [ ]:
scores2 = model_final.evaluate([x1_val, x2_val, x3_val],
                        y_val,
                        verbose=1
                        )
plt.plot(history.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')

# Визуализация точности на проверочной выборке
plt.plot(history.history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')

# Отрисовка подписей осей
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')

# Отрисовка легенды
plt.legend()

# Вывод графика
plt.show()


plt.plot(history.history['loss'],
         label='Ошибка на обучающем наборе')

plt.plot(history.history['val_loss'],
         label='Ошибка на проверочном наборе')

plt.xlabel('Эпоха обучения')
plt.ylabel('Ошибка')

plt.legend()

plt.show()